## 模組導入與環境套件安裝

In [ ]:
# 安裝必要的套件
!pip install langchain langchain-openai langchain-community chromadb openai

In [ ]:
# 導入核心模組
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import os
import json
from datetime import datetime

# 設定 OpenAI API 金鑰
os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"

print("環境準備完成，開始建立 RAG 系統...")


### 建立向量知識庫

In [ ]:
# 重建或載入向量知識庫
def setup_vector_store():
    """設定向量資料庫"""
    
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    
    # 嘗試載入現有的資料庫
    try:
        vector_store = Chroma(
            persist_directory="./chroma_db_demo",
            embedding_function=embeddings,
            collection_name="ai_knowledge_base"
        )
        
        # 檢查是否有資料
        if vector_store._collection.count() > 0:
            print(f"成功載入現有向量資料庫，包含 {vector_store._collection.count()} 個文件")
            return vector_store
        else:
            print("現有資料庫為空，將創建新的資料庫...")
    except:
        print("未找到現有資料庫，將創建新的資料庫...")
    
    # 創建新的資料庫
    sample_documents = [
        Document(
            page_content="""
深度學習是機器學習的一個子集，它使用多層神經網路來模擬人腦的決策過程。
深度學習的主要架構包括：

1. 卷積神經網路（CNN）：專門用於處理圖像數據，通過卷積層檢測局部特徵。
CNN 在電腦視覺任務中表現優異，如圖像分類、目標檢測和圖像分割。

2. 循環神經網路（RNN）：適合處理序列數據，如時間序列或自然語言。
LSTM 和 GRU 是 RNN 的改進版本，解決了長期依賴問題。

3. Transformer：徹底改變了自然語言處理領域，是 GPT 和 BERT 的基礎。
Transformer 使用注意力機制來捕捉序列中的長距離依賴關係。
            """,
            metadata={
                "source": "deep_learning_architectures.md",
                "category": "深度學習",
                "level": "進階",
                "topics": ["CNN", "RNN", "Transformer"]
            }
        ),
        Document(
            page_content="""
機器學習的資料預處理是模型成功的關鍵步驟。主要包括：

1. 資料清理：移除重複值、處理缺失值、識別和處理異常值。
缺失值可以通過刪除、填補（平均值、中位數、眾數）或預測來處理。

2. 特徵工程：創建新特徵、特徵選擇、特徵縮放和特徵轉換。
特徵縮放包括標準化（z-score）和正規化（min-max scaling）。

3. 資料分割：將資料分為訓練集、驗證集和測試集。
典型的分割比例是 70% 訓練、15% 驗證、15% 測試。

4. 資料增強：特別在電腦視覺中，通過旋轉、翻轉、縮放等方式增加資料多樣性。
            """,
            metadata={
                "source": "data_preprocessing.md",
                "category": "機器學習",
                "level": "中級",
                "topics": ["資料清理", "特徵工程", "資料分割"]
            }
        ),
        Document(
            page_content="""
人工智慧倫理是確保 AI 技術負責任發展的重要領域。主要考量包括：

1. 演算法公平性：確保 AI 系統不會產生歧視性結果。
需要檢測和減少訓練資料中的偏見，並評估模型對不同群體的影響。

2. 透明度和可解釋性：使用者有權理解 AI 系統如何做出決策。
可解釋 AI（XAI）技術幫助理解模型的決策過程。

3. 隱私保護：在使用個人資料訓練模型時保護個人隱私。
差分隱私和聯邦學習是重要的隱私保護技術。

4. 責任歸屬：當 AI 系統造成傷害時，如何確定責任主體。
需要建立清楚的責任鏈和問責機制。
            """,
            metadata={
                "source": "ai_ethics.md",
                "category": "AI 倫理",
                "level": "專業",
                "topics": ["公平性", "透明度", "隱私", "責任歸屬"]
            }
        ),
        Document(
            page_content="""
自然語言處理（NLP）讓機器能夠理解和生成人類語言。核心技術包括：

1. 文本預處理：分詞、詞性標註、命名實體識別、語法分析。
中文 NLP 面臨分詞挑戰，需要專門的分詞工具如 jieba。

2. 詞嵌入：將詞彙轉換為向量表示，捕捉語義關係。
Word2Vec、GloVe 和 FastText 是傳統的詞嵌入方法。
BERT 和 GPT 等預訓練模型提供了上下文相關的詞嵌入。

3. 語言模型：預測下一個詞或評估文本序列的概率。
大型語言模型如 GPT-4 展現了強大的文本生成和理解能力。

4. 應用領域：機器翻譯、情感分析、問答系統、文本摘要、對話系統。
            """,
            metadata={
                "source": "nlp_fundamentals.md",
                "category": "自然語言處理",
                "level": "中級",
                "topics": ["文本預處理", "詞嵌入", "語言模型", "NLP應用"]
            }
        ),
        Document(
            page_content="""
電腦視覺使機器能夠理解和解釋視覺資訊。主要任務包括：

1. 圖像分類：識別圖像中的主要物體或場景。
CNN 是圖像分類的標準架構，ResNet、VGG、Inception 是經典模型。

2. 目標檢測：在圖像中定位和識別多個物體。
YOLO、R-CNN 系列是目標檢測的代表性方法。

3. 圖像分割：將圖像分割成有意義的區域或像素級標註。
語義分割標註每個像素的類別，實例分割區分不同實例。

4. 人臉識別：識別和驗證人臉身份。
涉及人臉檢測、人臉對齊、特徵提取和相似度計算。

5. 光學字符識別（OCR）：從圖像中提取文字資訊。
現代 OCR 結合了深度學習和傳統圖像處理技術。
            """,
            metadata={
                "source": "computer_vision.md",
                "category": "電腦視覺",
                "level": "進階",
                "topics": ["圖像分類", "目標檢測", "圖像分割", "人臉識別", "OCR"]
            }
        )
    ]
    
    vector_store = Chroma.from_documents(
        documents=sample_documents,
        embedding=embeddings,
        persist_directory="./chroma_db_demo",
        collection_name="ai_knowledge_base"
    )
    
    print(f"新向量資料庫建立完成，包含 {len(sample_documents)} 個文件")
    return vector_store

# 設定向量資料庫
vector_store = setup_vector_store()


### 不同配置

In [ ]:
# 配置不同的語言模型
def setup_language_models():
    """設定不同配置的語言模型"""
    
    models = {
        "gpt-3.5-turbo": ChatOpenAI(
            model_name="gpt-3.5-turbo",
            temperature=0.1,  # 較低溫度確保回答一致性
            max_tokens=500    # 限制回答長度
        ),
        "gpt-3.5-turbo-creative": ChatOpenAI(
            model_name="gpt-3.5-turbo",
            temperature=0.7,  # 較高溫度增加創造性
            max_tokens=800
        ),
        "gpt-4": ChatOpenAI(
            model_name="gpt-4",
            temperature=0.2,  # 平衡一致性和靈活性
            max_tokens=600
        )
    }
    
    print("語言模型配置完成:")
    for name, model in models.items():
        print(f"- {name}: temperature={model.temperature}, max_tokens={model.max_tokens}")
    
    return models

language_models = setup_language_models()


### 自訂模組

In [ ]:
# 設計不同風格的提示模板
def create_prompt_templates():
    """創建不同風格的提示模板"""
    
    templates = {
        "professional": """你是一位專業的 AI 技術專家，請根據以下提供的資料來回答問題。

相關資料：
{context}

問題：{question}

回答指南：
1. 請基於提供的資料給出準確、專業的回答
2. 如果資料中沒有足夠資訊，請明確說明
3. 請組織清晰的回答結構，使用適當的技術術語
4. 如果可能，請提供具體的例子或應用場景

回答：""",

        "educational": """你是一位耐心的 AI 技術導師，正在為學生解答問題。

學習資料：
{context}

學生問題：{question}

教學要求：
1. 用淺顯易懂的語言解釋複雜概念
2. 提供具體例子幫助理解
3. 指出重要的學習要點
4. 如果資料不完整，建議進一步學習的方向
5. 保持鼓勵和積極的語調

教學回答：""",

        "concise": """基於以下資料回答問題，要求簡潔明確。

資料：
{context}

問題：{question}

要求：直接回答，條理清晰，避免冗長說明。

回答：""",

        "analytical": """你是一位 AI 研究分析師，請進行深度分析。

研究資料：
{context}

分析問題：{question}

分析框架：
1. 核心概念解析
2. 技術細節分析  
3. 優勢與限制
4. 實際應用案例
5. 發展趨勢預測

詳細分析："""
    }
    
    prompt_objects = {}
    for name, template in templates.items():
        prompt_objects[name] = PromptTemplate(
            template=template,
            input_variables=["context", "question"]
        )
    
    print("提示模板創建完成:")
    for name in templates.keys():
        print(f"- {name}: 適合{['專業諮詢', '教學輔導', '快速查詢', '深度分析'][list(templates.keys()).index(name)]}場景")
    
    return prompt_objects

prompt_templates = create_prompt_templates()


### 建立 RetrievalQA Chain

# 建立完整的 RAG 系統
class ComprehensiveRAGSystem:
    """綜合 RAG 系統"""
    
    def __init__(self, vector_store, language_models, prompt_templates):
        self.vector_store = vector_store
        self.language_models = language_models
        self.prompt_templates = prompt_templates
        self.qa_chains = {}
        self.setup_qa_chains()
    
    def setup_qa_chains(self):
        """設定各種 QA 鏈組合"""
        
        print("正在建立 RAG 鏈組合...")
        
        # 創建檢索器
        retriever = self.vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 3}  # 檢索最相關的 3 個文件
        )
        
        # 為每個模型和提示模板組合建立 QA 鏈
        for model_name, llm in self.language_models.items():
            for prompt_name, prompt in self.prompt_templates.items():
                chain_name = f"{model_name}_{prompt_name}"
                
                qa_chain = RetrievalQA.from_chain_type(
                    llm=llm,
                    chain_type="stuff",  # 將所有檢索文件合併
                    retriever=retriever,
                    chain_type_kwargs={"prompt": prompt},
                    return_source_documents=True,  # 返回來源文件
                    verbose=False  # 設為 True 可以看到詳細過程
                )
                
                self.qa_chains[chain_name] = qa_chain
        
        print(f"成功建立 {len(self.qa_chains)} 個 RAG 鏈組合")
        
        # 創建一個預設的最佳組合
        self.default_chain = self.qa_chains["gpt-3.5-turbo_professional"]
        print("預設使用：gpt-3.5-turbo + professional 提示模板")
    
    def ask_question(self, question, chain_name=None, show_sources=True):
        """向 RAG 系統提問"""
        
        if chain_name is None:
            chain = self.default_chain
            chain_name = "預設鏈"
        else:
            if chain_name not in self.qa_chains:
                print(f"錯誤：找不到鏈 '{chain_name}'")
                print(f"可用的鏈：{list(self.qa_chains.keys())}")
                return None
            chain = self.qa_chains[chain_name]
        
        print(f"\n=== 使用 {chain_name} 回答問題 ===")
        print(f"問題：{question}")
        print("-" * 60)
        
        try:
            # 執行查詢
            result = chain({"query": question})
            
            print("回答：")
            print(result["result"])
            
            if show_sources:
                print(f"\n參考來源：")
                for i, doc in enumerate(result["source_documents"]):
                    print(f"\n來源 {i+1}：{doc.metadata['source']}")
                    print(f"類別：{doc.metadata['category']} | 級別：{doc.metadata['level']}")
                    print(f"內容摘要：{doc.page_content[:100]}...")
            
            return result
            
        except Exception as e:
            print(f"查詢出錯：{str(e)}")
            return None
    
    def compare_responses(self, question, chain_names=None):
        """比較不同鏈的回答"""
        
        if chain_names is None:
            # 預設比較幾個代表性組合
            chain_names = [
                "gpt-3.5-turbo_professional",
                "gpt-3.5-turbo_educational", 
                "gpt-3.5-turbo_concise"
            ]
        
        print(f"\n=== 比較不同 RAG 配置的回答 ===")
        print(f"問題：{question}")
        print("=" * 80)
        
        results = {}
        for chain_name in chain_names:
            if chain_name in self.qa_chains:
                print(f"\n【{chain_name}】")
                result = self.ask_question(question, chain_name, show_sources=False)
                results[chain_name] = result
                print("-" * 40)
        
        return results
    
    def get_system_info(self):
        """獲取系統資訊"""
        info = {
            "vector_store_documents": self.vector_store._collection.count(),
            "available_models": list(self.language_models.keys()),
            "available_prompts": list(self.prompt_templates.keys()),
            "total_qa_chains": len(self.qa_chains)
        }
        
        print("\n=== RAG 系統資訊 ===")
        print(f"向量資料庫文件數：{info['vector_store_documents']}")
        print(f"可用語言模型：{', '.join(info['available_models'])}")
        print(f"可用提示模板：{', '.join(info['available_prompts'])}")
        print(f"RAG 鏈組合總數：{info['total_qa_chains']}")
        
        return info

# 建立綜合 RAG 系統
rag_system = ComprehensiveRAGSystem(vector_store, language_models, prompt_templates)
system_info = rag_system.get_system_info()

---

## 實戰驗證 RAG 系統

### 測試基礎問答能力

In [ ]:
# 測試基礎問答能力
def test_basic_qa():
    """測試基礎問答功能"""
    
    basic_questions = [
        "什麼是深度學習？它有哪些主要架構？",
        "在機器學習中，資料預處理包括哪些步驟？",
        "AI 倫理有哪些主要考量？",
        "自然語言處理的核心技術是什麼？",
        "電腦視覺的主要任務有哪些？"
    ]
    
    print("=== 基礎問答能力測試 ===")
    
    for i, question in enumerate(basic_questions, 1):
        print(f"\n測試 {i}/5")
        result = rag_system.ask_question(question)
        
        if result:
            # 簡單的品質評估
            answer_length = len(result["result"])
            num_sources = len(result["source_documents"])
            
            print(f"回答長度：{answer_length} 字符")
            print(f"參考來源：{num_sources} 個文件")
            
            # 檢查回答是否包含關鍵資訊
            if "不知道" in result["result"] or "沒有資訊" in result["result"]:
                print("⚠️  系統表示資訊不足")
            else:
                print("✅ 系統提供了回答")
        
        print("\n" + "="*60)

# 執行基礎測試
test_basic_qa()


### 不同風格比較與回答

In [ ]:
# 比較不同提示模板的回答風格
def test_response_styles():
    """測試不同回答風格"""
    
    test_question = "什麼是卷積神經網路（CNN）？它適合處理什麼類型的問題？"
    
    print("=== 不同回答風格比較測試 ===")
    
    style_chains = [
        "gpt-3.5-turbo_professional",
        "gpt-3.5-turbo_educational",
        "gpt-3.5-turbo_concise",
        "gpt-3.5-turbo_analytical"
    ]
    
    results = rag_system.compare_responses(test_question, style_chains)
    
    # 分析不同風格的特點
    print("\n=== 回答風格分析 ===")
    for chain_name, result in results.items():
        if result:
            answer = result["result"]
            style = chain_name.split("_")[1]
            
            print(f"\n{style} 風格特點：")
            print(f"- 長度：{len(answer)} 字符")
            print(f"- 結構化程度：{'高' if '1.' in answer or '•' in answer else '中'}")
            print(f"- 技術深度：{'深' if len([w for w in ['架構', '演算法', '模型', '訓練'] if w in answer]) > 2 else '中'}")

test_response_styles()


### edge-case 測試

In [ ]:
# 測試邊界情況
def test_edge_cases():
    """測試系統在邊界情況下的表現"""
    
    edge_cases = [
        {
            "question": "量子計算與機器學習的結合有什麼前景？",
            "description": "超出知識庫範圍的問題"
        },
        {
            "question": "今天天氣如何？",
            "description": "完全無關的問題"
        },
        {
            "question": "CNN",
            "description": "極簡問題"
        },
        {
            "question": "請詳細解釋深度學習的歷史發展、現狀、未來趨勢、技術挑戰、應用領域、商業價值以及對社會的影響",
            "description": "過於複雜的複合問題"
        },
        {
            "question": "",
            "description": "空問題"
        }
    ]
    
    print("=== 邊界情況測試 ===")
    
    for i, case in enumerate(edge_cases, 1):
        print(f"\n測試 {i}: {case['description']}")
        print(f"問題：{case['question'] if case['question'] else '(空問題)'}")
        print("-" * 40)
        
        if not case['question']:
            print("跳過空問題測試")
            continue
        
        result = rag_system.ask_question(case['question'], show_sources=False)
        
        if result:
            answer = result["result"]
            
            # 分析回答品質
            if any(phrase in answer.lower() for phrase in ['不知道', '沒有資訊', '無法回答', '資料中沒有']):
                print("✅ 系統正確識別出資訊不足")
            elif case['description'] in ["超出知識庫範圍的問題", "完全無關的問題"]:
                print("⚠️  系統可能回答了超出知識範圍的問題")
            else:
                print("✅ 系統提供了合理回答")
        else:
            print("❌ 系統執行出錯")
        
        print("\n" + "="*50)

test_edge_cases()


### 檢索品質分析

In [ ]:
# 分析檢索品質
def analyze_retrieval_quality():
    """分析檢索系統的品質"""
    
    print("=== 檢索品質分析 ===")
    
    # 測試不同的檢索參數
    test_questions = [
        "深度學習的 Transformer 架構",
        "如何處理機器學習中的缺失值？",
        "AI 系統的公平性問題"
    ]
    
    retrieval_configs = [
        {"k": 1, "description": "檢索1個文件"},
        {"k": 3, "description": "檢索3個文件"},
        {"k": 5, "description": "檢索5個文件"}
    ]
    
    for question in test_questions:
        print(f"\n問題：{question}")
        print("-" * 50)
        
        for config in retrieval_configs:
            print(f"\n{config['description']}：")
            
            # 創建臨時檢索器
            temp_retriever = vector_store.as_retriever(
                search_type="similarity",
                search_kwargs={"k": config["k"]}
            )
            
            # 執行檢索
            docs = temp_retriever.get_relevant_documents(question)
            
            print(f"檢索到 {len(docs)} 個文件片段")
            
            # 分析檢索結果的多樣性
            categories = set(doc.metadata['category'] for doc in docs)
            sources = set(doc.metadata['source'] for doc in docs)
            
            print(f"涵蓋類別：{', '.join(categories)}")
            print(f"來源文件：{len(sources)} 個")
            
            # 顯示相關性
            for i, doc in enumerate(docs):
                content_preview = doc.page_content[:60].replace('\n', ' ')
                print(f"  {i+1}. [{doc.metadata['category']}] {content_preview}...")
        
        print("\n" + "="*60)

analyze_retrieval_quality()


### 系統性能評估

In [ ]:
# 評估系統整體性能
def evaluate_system_performance():
    """評估 RAG 系統的整體性能"""
    
    print("=== RAG 系統性能評估 ===")
    
    import time
    
    # 準備評估問題集
    evaluation_questions = [
        "深度學習有哪些主要架構？",
        "資料預處理的步驟是什麼？",
        "AI 倫理的重要考量有哪些？",
        "NLP 的核心技術包括什麼？",
        "電腦視覺的主要任務是什麼？"
    ]
    
    performance_results = {
        "response_times": [],
        "answer_lengths": [],
        "source_counts": [],
        "success_rate": 0
    }
    
    successful_responses = 0
    
    print("開始性能測試...")
    
    for i, question in enumerate(evaluation_questions, 1):
        print(f"\n測試 {i}/{len(evaluation_questions)}: {question[:30]}...")
        
        start_time = time.time()
        result = rag_system.ask_question(question, show_sources=False)
        end_time = time.time()
        
        response_time = end_time - start_time
        performance_results["response_times"].append(response_time)
        
        if result and result["result"]:
            successful_responses += 1
            answer_length = len(result["result"])
            source_count = len(result["source_documents"])
            
            performance_results["answer_lengths"].append(answer_length)
            performance_results["source_counts"].append(source_count)
            
            print(f"✅ 回應時間：{response_time:.2f}秒，回答長度：{answer_length}字符")
        else:
            print(f"❌ 回應失敗")
    
    # 計算統計資料
    performance_results["success_rate"] = successful_responses / len(evaluation_questions)
    
    print(f"\n=== 性能統計 ===")
    print(f"成功率：{performance_results['success_rate']:.1%}")
    
    if performance_results["response_times"]:
        avg_response_time = sum(performance_results["response_times"]) / len(performance_results["response_times"])
        print(f"平均回應時間：{avg_response_time:.2f}秒")
    
    if performance_results["answer_lengths"]:
        avg_answer_length = sum(performance_results["answer_lengths"]) / len(performance_results["answer_lengths"])
        print(f"平均回答長度：{avg_answer_length:.0f}字符")
    
    if performance_results["source_counts"]:
        avg_source_count = sum(performance_results["source_counts"]) / len(performance_results["source_counts"])
        print(f"平均參考來源：{avg_source_count:.1f}個文件")
    
    return performance_results

performance_data = evaluate_system_performance()
